# Type structure and interface of unitcell objects

The most defining feature of a lattice is the periodic arrangement of **sites**, connected by **bonds**. However, not every site can be mapped onto every other by translation of Bravais lattices, but one can define groups of sites, that can be mapped onto each other, so called **unitcells** of the lattice. Similar to site and bond, unitcells therefore have their own type in `LatticePhysics.jl`, equipped with an abstract supertype as well as a shared interface.

The following tutorial aims on providing an overview over the type hierachy of unitcells, introducing the abstract type `AbstractUnitcell`, as well as discussing the interface functions for this abstract type by showing examples for the concrete struct `Unitcell`.


In [ ]:
using LatPhysBase

### Abstract type `AbstractUnitcell{S,B}`

The abstract type that is supertype to all unitcell implementations is `AbstractUnitcell`, parametric in two parameters `S` and `B`. These two parameters are the site type `S`, itself subtype of `AbstractSite`, as well as the bond type `B`, subtype of `AbstractBond`.

Being parametric in these two types already is sufficient to dispatch unitcell objects based on
- real space dimension
- Bravais lattice dimension
- label types of sites and bonds

This dispatch behavior allows for a compact but yet powerful solution in unifying syntax, e.g. `plotLattice` instead of two functions `plotLattice2D` and `plotLattice3D`.

In [ ]:
# this is a correct suptyping of unitcell types
AbstractUnitcell{ Site{String,2}, Bond{String,2} } <: AbstractUnitcell

In [ ]:
# correct, since both label types are the same and subtype of string
AbstractUnitcell{ Site{String,2}, Bond{String,2} } <: AbstractUnitcell{S,B} where {L,D,N,S<:AbstractSite{L,D},B <: AbstractBond{L,N}}

In [ ]:
# not correct, since real space dimension is not matching
AbstractUnitcell{ Site{String,2}, Bond{String,2} } <: AbstractUnitcell{S,B} where {L,S<:AbstractSite{L,3},B}

### Concrete type `Unitcell{S,B} <: AbstractUnitcell{S,B}`

For all explicit `Unitcell` and `Lattice` objects, a concrete unitcell type is needed. Although every user could in principle implement its own concrete unitcell type, a default implementation is provided within `LatticePhysics.jl`, the `mutable struct Unitcell` implementation.

`Unitcell` should be regarded as a subtype of `AbstractUnitcell` and can therefore be instatiated so it should be used whenever a concrete implementation is needed, i.e. whenever objects are created.

In [ ]:
Unitcell{ Site{String,2}, Bond{String,2} } <: AbstractUnitcell{ Site{String,2}, Bond{String,2} }

Note: Despite seemingly being fixed to constructing objects of concrete type `Unitcell{ Site{.}, Bond{.} }`, the parametrization of `Unitcell` allows for custom site and bond types to be inserted instead. For a user defined site type `MySite{L,D} <: AbstractSite{L,D}` (which might store more information than position and label), the type `Unitcell{ MySite{.}, Bond{.} }` can be constructed!




### Interface of `AbstractUnitcell` - how to access information correctly

In order to gain a benefit from having an abstract supertype `AbstractUnitcell`, one has to define a common interface that all concrete unitcell types that are subtype of `AbstractUnitcell` have to implement. Then one can simply call this interface for a given (but unknown) unitcell object that is subtype of `AbstractUnitcell` and rely on getting the correct results.

The interface of `AbstractUnitcell` contains various functions, which are explained in the following.


##### 1. The constructor

The constructor interface is used to create new objects of a specified unitcell type. It is therefore demanding to pass the type explicitly, as well as further information regarding the unitcell. 

The constructor syntax is
```
function newUnitcell(
            ::Type{U},
            lattice_vectors :: Vector{<:Vector{<:Real}},
            sites           :: Vector{S},
            bonds           :: Vector{B}
        ) :: U where{...}
```
This interface is overwritten by the concrete unitcell type `Unitcell` to
```
function newUnitcell(
            :: Type{Unitcell{S,B}},
            lattice_vectors :: Vector{<:Vector{<:Real}},
            sites           :: Vector{S},
            bonds           :: Vector{B}
        ) :: Unitcell{S,B} where {D,N,LS,LB,S<:AbstractSite{LS,D},B<:AbstractBond{LB,N}}
```


*Note: If you want a unitcell for your calculations, the chances are high that the unitcell is already implemented with all site positions and bonds set correctly. For these pre-implemented unitcells, check the pre-implemented unitcells tutorial ([[notebook](https://github.com/janattig/LatticePhysics_Tutorials/blob/master/basics/unitcells/pre-implemented_unitcells.ipynb)]).*

Using the constructor in constructing a unitcell object could work as follows:

In [ ]:
# list of sites
site_list = Site{String,2}[
    newSite(Site{String,2}, [0,0], "mysite")
]

# list of bonds
bond_list = Bond{String,2}[
    newBond(Bond{String,2}, 1, 1, "mybond", (1,0)),
    newBond(Bond{String,2}, 1, 1, "mybond", (-1,0)),
    newBond(Bond{String,2}, 1, 1, "mybond", (0,1)),
    newBond(Bond{String,2}, 1, 1, "mybond", (0,-1))
]

# list of lattice vectors
lattice_vectors = Vector{Float64}[
        Float64[1, 0],
        Float64[0, 1]
    ]

# construct the unitcell
uc = newUnitcell(
    Unitcell{Site{String,2},Bond{String,2}},
    lattice_vectors,
    site_list,
    bond_list
)

##### 2. Setter and getter of sites, bonds and lattice vectors

